### Turbo Analyzer script for running multiple files at once
A csv file will be stored in '../results/<EXPERIMENT>.csv'.
The turbo analyzer can also evaluate the average of a specific experiment, i.e.,
you run an experiment with the same parameters multiple times and store them in
a sub-folder of the targeted folder. 

The folder should have the same naming format as normal log files, but of course
without the ".txt" suffix. Files inside this folder can have any name, but 
must be suffixed with ".txt".

You will find the results printed as a dataframe at the bottom.

In [1]:
# Fill in the name of the experiment folder in logs/, .e.g 'SR1'
EXPERIMENT = "rtx-fec-test"

In [2]:
import os
import glob
import traceback
from io import StringIO
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from numpy import NaN
from functions import * # import functions from the .py file
from pylab import rcParams
rcParams["figure.figsize"] = 16, 4

clock = 125
logdir = f"./logs/{EXPERIMENT}"
experiments_data = []

print(f"Reading experiment folder '{logdir}'...")
for idx, path in enumerate(glob.glob(f"{logdir}/*")):
    log_paths = []
    # We have multiple log files for this configuration and we want to 
    # read them all and get the average. 
    if os.path.isdir(path):
        log_paths = glob.glob(f"{path}/*.txt")
    elif path.endswith(".txt"):
        log_paths.append(path)

    if len(log_paths) == 0:
        continue

    filename = os.path.basename(path)
    params = filename.removesuffix(".txt").split("_")

    use_fec = params[5].lower() == "fec" if len(params) >= 6 else False
    use_retransmission = False
    if len(params) > 6 and use_fec:
        use_retransmission = params[6].lower() == "rtx"

    dfs = [readfile(f, USE_RETRANSMISSION=use_retransmission) for f in log_paths]        

    experiments_data.append({
        "file": filename,
        "d2":int(params[0]), # tag to receiver (cm)
        "d1":int(params[1]), # carrier to tag (cm)
        "a": int(params[2]),
        "b": int(params[3]),
        "baud": int(params[4].replace('k', '000')),
        "delta_fsk": ((clock/int(params[3]) - clock/int(params[2])) * 1000)/2, # khz
        "file_delay_s": None,
        "bit_reliability": None,
        "dis_metric": None,
        "etx": None,
        "misses": None,
        "avg_rssi": None,
        "ecc": params[5].lower() == "ecc" if len(params) >= 6 else False,
        "fec": use_fec,
        "retransmission": use_retransmission, 
        "samples": len(dfs),
        "error": False,
        "dfs": dfs,
    })

def avg(lst):
    if len(lst) == 0:
        return 0
    return sum(lst) / len(lst)

def analyze():
    errors = []
    experiments = pd.DataFrame()

    # codes_hex = [format(int("".join([str(bit) for bit in code]), base=2), "04X") for code in walsh_codes]
    # print(codes_hex)
    # print(walsh_codes)

    for idx, data in enumerate(experiments_data):
        etx_lst = []
        misses_lst = []
        file_delay_s_lst = []
        bit_reliability_lst = []
        dis_metric_lst = []
        rssi_lst = []
        experiment_error = False
        use_ecc = data["ecc"]
        use_fec = data["fec"]

        for df in data["dfs"]:
            try:
                PAYLOADSIZE = 14
                NUM_16RND = int((PAYLOADSIZE-2) // 2) # how many 16 bits random number included in each frame
                if use_ecc:
                    PAYLOADSIZE = 12*3+2
                    NUM_16RND = int(((PAYLOADSIZE-2)/3) // 2) # how many 16 bits random number included in each frame
                elif use_fec:
                    PAYLOADSIZE = 6
                    # 4 packets needed to get a full sample, but we index by single 16-bit samples.
                    NUM_16RND = 1
                # if PAYLOADSIZE % 2 != 0:
                #     print("Alarm! the payload size is not even.")

                MAX_SEQ = 256 # (decimal) maximum seq number defined by the length of the seq, the length of seq is 1B

                payload_lens = set()
                expected_payload_len = (PAYLOADSIZE*3)-1
                for x in df.payload:
                    payload_lens.add(len(x))

                # delete the corrupted packets
                df = df[df.payload.apply(lambda x: len(x)==expected_payload_len)]
                df.reset_index(inplace=True)

                if len(df) == 0:
                    raise Exception(f"All packets are corrupt! Expected payload length of {expected_payload_len}, but got {payload_lens}")

                # replace the resettable seq number to unresettable, for the purpose of reliability calculation
                (test, test_rtx) = replace_seq(df.copy(), MAX_SEQ=256)
                test.rename(columns = {'seq':'old_seq', 'new_seq':'seq'}, inplace = True)
                test_rtx.rename(columns = {'seq':'old_seq', 'new_seq':'seq'}, inplace = True)

                file_delay = df.time_rx[len(df) - 1] - df.time_rx[0]
                file_delay_s = np.timedelta64(file_delay, "ms").astype(int) / 1000

                ber, error, etx, misses = compute_ber(test, test_rtx, PACKET_LEN=NUM_16RND*2, MAX_SEQ=MAX_SEQ, USE_ECC=use_ecc, USE_FEC=use_fec)
                bit_reliability = (1 - ber) * 100
                dis_carrier_tag = data["d1"] / 100 # cm -> m
                dis_tag_rx = data["d2"] / 100 # cm -> m
                dis_metric = dis_carrier_tag**2 * dis_tag_rx**2
                rssi = sum([x for x in df['rssi']])/len(df)

                etx_lst.append(etx)
                misses_lst.append(misses)
                file_delay_s_lst.append(file_delay_s)
                bit_reliability_lst.append(bit_reliability)
                dis_metric_lst.append(dis_metric)
                rssi_lst.append(rssi)
            except Exception as e:
                experiment_error = True
                errors.append((data["file"], "".join(traceback.format_exception(type(e), e, e.__traceback__))))

        avg_etx = avg(etx_lst)
        avg_misses = avg(misses_lst)
        avg_file_delay_s = avg(file_delay_s_lst)
        avg_bit_reliability = avg(bit_reliability_lst)
        avg_dis_metric = avg(dis_metric_lst)
        avg_rssi = avg(rssi_lst)

        del data["dfs"] # do not save log file dataframes to CSV
        experiments = pd.concat([pd.DataFrame(data, index=[idx]), experiments])
        experiments.loc[idx, "etx"] = avg_etx
        experiments.loc[idx, "misses"] = avg_misses
        experiments.loc[idx, "file_delay_s"] = avg_file_delay_s
        experiments.loc[idx, "bit_reliability"] = avg_bit_reliability
        experiments.loc[idx, "dis_metric"] = avg_dis_metric
        experiments.loc[idx, "avg_rssi"] = avg_rssi
        experiments.loc[idx, "error"] = experiment_error

        if not experiment_error:
            metrics = [avg_file_delay_s, avg_bit_reliability, avg_dis_metric]
            radar_plot(metrics, title=f"{data['file']} - Avg. of {data['samples']} sample(s)")

    if len(errors) > 0:
        print(f"{len(errors)} out of {len(experiments)} failed:")
        for (file, error) in errors:
            print(f"{file}:\n{error}")
        print("")

    print(experiments.to_string())

    filepath = Path(f"./results/{EXPERIMENT}.csv")  
    print(f"Saving experiment data to {str(filepath)}...")
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    experiments.to_csv(filepath, index=False)
    print("Successfully saved.")

if len(experiments_data) > 0:
    analyze()
else:
    print(f"No log files found in '{EXPERIMENT}'.")

Reading experiment folder './logs/rtx-fec-test'...
1 out of 1 failed:
50_5_54_52_42k_fec_rtx:
Traceback (most recent call last):
  File "/var/folders/j0/5pzb_3w57tj__yc0ry0gzczh0000gn/T/ipykernel_33691/641085485.py", line 113, in analyze
    raise Exception(f"All packets are corrupt! Expected payload length of {expected_payload_len}, but got {payload_lens}")
Exception: All packets are corrupt! Expected payload length of 14, but got {17}


                     file  d2  d1   a   b   baud  delta_fsk file_delay_s bit_reliability dis_metric etx misses avg_rssi    ecc   fec  retransmission  samples  error
0  50_5_54_52_42k_fec_rtx  50   5  54  52  42000   44.51567            0               0          0   0      0        0  False  True            True        1   True
Saving experiment data to results/rtx-fec-test.csv...
Successfully saved.
